<a href="https://colab.research.google.com/github/decoz/mlclass/blob/master/ch2_Mnist_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mnist Data 로 학습하기

MNIST database 란 (Modified National Institute of Standards and Technology database) 의 약자로서 미국에서 사람의 손글씨 학습을 위해 수집된 숫자 손글씨의 모음데이터이다. 원래는 Nist 데이터라고 불렸는데 이를 머신러닝에 적합하게 수정함으로서 Modified NIST 의 약자로 MNIST라 불린다.  주로 28x28 의 작은 60000 개의 학습이미지와 10000 개의 테스트 이미지로 구성되어있으며 각각에 대한 레이블 값도 제공된다. 

요즘은 28x28 로 구성된 작은 학습이미지 샘플 자체를 MNIST라고 부르기도 한다. 


## keras 에서 mnist 데이터 불러오기

먼저 keras.datasets.mnist 를 불러온다. 

In [0]:
from keras.datasets import fashion_mnist

mnist.load_data 를 호출하면 train, test 두개의 데이터 셋을 리턴한다. 

In [0]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data() 

각각의 형태를 확인하면.train_images ,  test_images 는 28,28 의 이차원배열들이 같이 모여서 구성된 3차원배열의 형태를 하고 있으며 train_labels , test_labels 는 각 이미지가 어떤 숫자를 표현하는지가 0~9의 숫자들의 배열로 구성되어있다. 


In [0]:
print( train_images.shape , test_images.shape)
print( train_labels.shape , test_labels.shape)

In [0]:
print(test_labels)
print(train_labels)

실제 데이터의 모습을 확인하는 방법은 plt.imshow 를 사용하면 된다. 다음은 6만개의 학습 이미지 중 첫번째 이미지의 모습을 그려준다. 

In [0]:
import matplotlib.pyplot as plt 
plt.imshow(train_images[1], cmap = 'gray')
plt.show()

실제로 train_labels[0] 의 값도 5로 되어있는걸 확인할 수 있다. 

In [0]:
train_labels[1]

## 모델 구성하기

MNiST는 비교적 심플한 문제이며 사실 대부분의 학습 알고리즘이 손쉽게 90% 이상의 정확도를 찍지만 그럼에도 99.5% 이상을 보이려면 cnn등의 복잡한 신경망이 사용되기도 하는 문제이다.  우리는 우리가 알고 있는 가장 간단한 단층 신경망을 사용하여 이 문제를 풀어보도록 하겠다. 

먼저 입력은 28x28 이므로 784개이며 출력은 0~9 까지의 10가지 클래스로 나뉘기 때문에 10 개의 출력 값을 갖도록하자. 이 문제는 데이터량은 크지만 문제의 복잡도는 크지 안기때문에 입력값의 활성함수로 relu 를 사용하도록 하겠다. 

In [0]:
import keras
from keras import models
from keras import layers
keras.backend.clear_session()
model = models.Sequential() 

model.add(layers.Dense(512, activation = 'relu', input_shape = (28*28,)))
model.add(layers.Dense(10, activation='softmax'))

In [0]:
model.compile(optimizer='rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

## 데이터 포맷 변환 

Dense 신경망은 1차원 배열만을 입력으로 받기 때문에 28 x 28 의 데이터는 784 개의 1차원 배열로 바꾸어 주어야 한다. 또한 0 ~ 255의 값도 이를 실수화시켜서 0~1까지의 입력값으로 정규화 해주는게 좋다. 

In [0]:
tr_data = train_images.reshape((60000, 28*28))
tr_data = tr_data.astype('float32') / 255

tst_data = test_images.reshape((10000, 28*28))
tst_data = tst_data.astype('float32') / 255

지난 챕터에서도 언급했지만 classification 신경망은 각각의 클래스(레이블)을 0 ~ 9 같은 연속값으로 처리하지 안는다.  0,1 을 [0,1] , [1,0] 으로 표현하듯이 0~9 까지도 예를 들어 9의 경우 [0,0,0,0,0,0,0,0,0,1] 로 표현해야한다. 그러므로 만일 [0,2,4] 라는 레이블값은 

<pre>
[
    [1,0,0,0,0,0,0,0,0,0],
    [0,0,1,0,0,0,0,0,0,0],
    [0,0,0,0,1,0,0,0,0,0]
]
</pre>

로 표현되어야 한다. 

물론 이를 코딩하는 것은 어렵지 안은 문제이지만 그래도 자주사용되는 기능은 죄다 함수로 제공해주는 파이썬답게 이런 함수도 이미 존재한다. 

    keras.utils.to_categorical( arr )
    
여기에 [2,4] 를 넣으면 다음과 같이 리턴한다. 


    
 

In [0]:
import numpy as np 
keras.utils.to_categorical( np.array([2,4]) )

이제 이를 이용해 학습용과 검증용 두 레이블을 변환해주도록 한다. 

In [0]:
from keras.utils import to_categorical
tr_labels = to_categorical(train_labels)
tst_labels = to_categorical(test_labels)
print(tr_labels)

In [0]:
#network.fit(train_images, train_lables, epochs=5, batch_size=128)
model.summary()


## 학습 후 정확도 테스트

이제 위에서 준비한 데이터 tr_data 와 tr_labels 를 사용해 학습을 수행한다. 

mnist 데이터는 매우 빠르게 수렴하는 쉬운 학습 예제중에 하나이다. 게다가 트레이닝 데이터가 풍부하므로 10회정도만 학습을 수행해도 실제로는 60만번의 수정이 이뤄지므로 대부분 99% 이상의 정확도에 수렴한다.

In [0]:
hist = model.fit(tr_data, tr_labels, epochs=10, batch_size=128)

이제 학습된 신경망에 테스트 데이터를 넣어서 테스트해보자. predict 를 이용해 직접할 수도 있겠지만 역시 이를 편하게 하기 위한 evaluate 함수를 제공해준다. 

In [0]:
test_loss, test_acc = model.evaluate(tst_data, tst_labels)
print("loss:",test_loss, "\naccuracy:", test_acc)

실제 이를 활용할 수 있나 알아보기 위해서 test 이미지 하나를 골라서 직접 그 출력을 살펴보도록하자. 여기서는 랜덤하게 919번째 데이터를 선택해보겠다. 

In [0]:
plt.imshow(test_images[218], cmap = 'gray')
plt.show() 

r =  model.predict([tst_data[np.newaxis,218]]) 
print(np.around(r,3))

print( np.argmax(r, axis = 1) )

<font color = 'red'>
    
###  연습문제 2-1 : 정확도 변화 보기 
    
위의 학습에서 10회동안 발생한 정확도의 변화를 그래프로 그려보세요 
    

In [0]:
## 연습문제 2-1을 위한 코드를 작성하세요 


<font color = 'red'>
    
###  연습문제 2-2 : 적합도 변화량 체크 
   
모델을 첨부터 다시 설정하고 epoch 를 1회씩 수행한 후  evaluate 를 수행하는 과정을 10회 반복하며 학습 정확도와 검증 정확도의 변화를 그래프로 그려보세요
    

In [0]:
## 연습문제 2-2 를 위한 코드를 작성하세요
 


<hr style="height:2px" > 

### 참고 : Mnist 데이터 뷰어 만들기

매번 하나씩 mnist 데이터를 확인하는건 답답한 일일 수 있다. 아래 간단한 부록 개념으로 mnist 데이터 이미지를 타일로 합쳐서 보여주는 코드를 작성해보았다. 

In [0]:
import math
def mnist_gallery(imgs, idx, n, wn):    
    w,h = imgs.shape[1],imgs.shape[2]        
    img = np.empty((0,w * wn))    
    hn = math.ceil(n / wn)
        
    limg = np.empty((h,0))    
    
    for i in range(wn * hn):
        if i < n : 
            limg = np.hstack((limg, imgs[idx+i] ))        
        else :
            limg = np.hstack((limg, np.zeros((h,w))))      
          
        if ( i + 1 ) % wn == 0 :
            img = np.vstack((img, limg))        
            limg = np.empty((h,0))
            
    return img 

사용법은 다음과 같다. 

    img = mnist_gallery( 이미지셋, 시작인덱스, 이미지 숫자, 한줄에 표기되는 이미지 갯수 )
    
만일 0번부터 25 개의 이미지를 5x5 개로 그리고 싶다면 다음과 같이 실행하면 된다. 

In [0]:
img = mnist_gallery(test_images, 0, 50, 5 )
plt.imshow(img, cmap = 'gray')
plt.show()


### 참고 2 : 특정 레이블 데이터(이미지) 추출 

만일 우리가 숫자 5의 이미지만 추출해 보고싶다면 어떻게 해야할까?  이를 위해서는 np.where 이라는 함수가 유용하다. 


In [0]:
arr = np.array([1,2,3,1,2])
w1 = np.where(arr == 1)
print( w1  ) 
print(arr[w1])

np.where( arr == 1 ) 은 arr 값이 1인 인덱스만을 뽑아서 출력해준다.  값이 아니라 인덱스 값이기에 1의 위치인 [0,3] 이 리턴된 것이다. 
이 결과([0,3]) 을 이것을 arr에 배열인덱스로 넣어주면 0,3 위치의 값인 1,1 이 출력되었음을 볼 수 있다. 

이를 응용하면 다음과 같이 전체 이미지에서 5의 이미지만을 뽑아낼 수 있다. 


In [0]:
tr5_idx = np.where( train_labels > 5 )
tr5_images = train_images[tr5_idx] 

img = mnist_gallery(tr5_images, 0, 25, 5 )
plt.imshow(img, cmap = 'gray')
plt.show()

<hr style="height:2px" > 

## 커스텀 이미지 판별

이쯤되면 실제 자신의 이미지를 이 모델에 적용해 예측을 해보고 싶은 생각이 들 것이다.  다음은 그림판으로 그려서 만든 간단한 이미지이다.

<img src = "https://github.com/decoz/mlclass/blob/master/images/4.png?raw=true" align = "left">


여러분들도 그림판에 간단한 숫자 이미지를 그려서 넣고 저장한 후에 이를 쥬피터 노트북 실행 폴더에 추가하자. 

간단한 이미지 로딩 및 출력는 matplotlib 로도 가능하지만 모델을 사용하려면 저 이미지를 28x28 사이즈로 축소하여야 한다. 이를 위해 되도록이면 추가 라이브러리 설치를 피하기 위해 간단한 PIL 라이브러리를 사용하도록 하겠다. 아래에는 위의 이미지를 불러들인 후에 28x28이미지로 변환하는 코드이다.  

### 이미지 로드후 크기 변환


In [0]:
from PIL import Image

im = Image.open("4.png").convert('L')
im = im.resize((28,28))
plt.imshow(im, cmap='gray')

### PIL 이미지를 numpy 배열로 변환

PIL 이미지에 image data 를 불러오는 함수는 getdata() 이다. 단 이 경우 데이터는 1차원 배열로 호출되기 때문에 만일 이를 2차원 배열로 변환하고 싶다면 image.size 인자값을 이용해 다음과 같이 생성할 수 있따. 

In [0]:
im = np.array(im.getdata()).reshape(im.size[0], im.size[1])

위의 이미지는 흰 배경에 검은 색 글씨이므로 학습 모델에 맞추기 위해선 반전이 필요하다. 다음의 코드를 이용해 반전시킬수 있다. 

    im = 255 - im
    
단 기왕이면 반전이 필요한지 전체 값중에 밝은 값과 어두운 값의 비율을 검사해서 해주면 더 사용하기 편한 코드가 가능할 것이다. 이를 위해 if 문의 조건으로 전체 평균값이 128이상인지를 체크하는 코드를 추가하도록 하겠다. 

In [0]:
if np.sum(im) > np.sum( im.shape[0] * im.shape[1] * 128 ) :
    im = 255 - im    
plt.imshow(im, cmap = 'gray')
plt.show()

이제 28,28 로 되어있는 데이터를 1,784 로 변환하여 모델로 예측을 수행해보자. 

In [0]:
    
data = im.reshape(1,28*28)
r = model.predict(data)
print("result num:",np.argmax(r))


<hr> 

여러 숫자 이미지 판별 

다음 코드는 여러 숫자가 1열로 들어있는 이미지를 읽어들여서 처리하는 코드이다. 

<img src = "https://github.com/decoz/mlclass/blob/master/images/t_nums2.png?raw=true" align = left > 




숫자를 컷트하기 위해 세로로 이미지의 합산값을 구한 후에 이를 커트 하고 양 옆에 여백의 이미지를 붙여주었다. 중간에 이미지를 resize 하기 위해 numpy -> iamge -> numpy 로 변환과정을 거치므로 조금 복잡하기 때문에 이런 코드로 가능하다는 참고정도로 돌려보면 좋을듯 하다. 


In [0]:

im = Image.open("t_nums2.png").convert('L')
im = np.array(im.getdata()).reshape(im.size[1], im.size[0])
if np.sum(im) > np.sum( im.shape[0] * im.shape[1] * 128 ) :
    im = 255 - im    
plt.imshow(im, cmap = 'gray')
plt.show()
vsum = np.sum(im, axis = 0)

thresh = 0


s = -1 
w,h = 28,28

inputs = np.empty((0,28*28))

for i in range( im.shape[1] ):
    if s >= 0 and vsum[i] <= thresh :         
        narr = im[:,s:i].copy()      
        pad = np.zeros( (narr.shape[0],3) ) 
        narr = np.hstack( (pad, narr, pad) ) 
        
        num = Image.fromarray(narr)
        num = np.array( num.resize((w,h)).getdata() )        
        inputs = np.vstack((inputs, num))
        print(inputs.shape)
        
        s = -1
    elif s < 0 and vsum[i] > thresh : 
        s = i

if s > 0 :
    areas += im[a:,s:i].copy()

output = model.predict(inputs)
result = np.argmax(output, axis = 1)
print(result)
#print(areas)                
            
#print(im.shape, vsum)